In [1]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture

In [2]:
df = pd.read_csv('../data/fashion/fashion-mnist_train.csv')

In [3]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
y = df['label']
del df['label']

In [6]:
%%time
gmm = GaussianMixture(n_components=10, covariance_type='diag').fit(df)

CPU times: user 1min 14s, sys: 15.9 s, total: 1min 30s
Wall time: 46.8 s


In [17]:
gmm.means_[0]

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,

In [13]:
%%time
gmm2 = GaussianMixture(n_components=10, covariance_type='full').fit(df)

CPU times: user 36min 1s, sys: 3min 45s, total: 39min 47s
Wall time: 22min 38s


In [18]:
gmm2.means_[0]

array([  1.48669453e-03,   1.42297905e-02,   7.47594965e-02,
         5.13759160e-01,   1.24605848e+00,   1.92907245e+00,
         3.06259075e+00,   7.11829413e+00,   1.58165096e+01,
         3.05525545e+01,   5.07010520e+01,   7.59895767e+01,
         8.81925975e+01,   7.80554665e+01,   7.72660318e+01,
         8.73075629e+01,   8.38470873e+01,   5.58844258e+01,
         3.40023503e+01,   1.92252763e+01,   8.82756714e+00,
         3.95258104e+00,   2.37616297e+00,   1.74877755e+00,
         1.13328585e+00,   4.69158317e-01,   1.15112641e-01,
         3.18577400e-03,   3.78045181e-02,   1.11289705e-01,
         3.76983256e-01,   2.44943151e+00,   4.76803391e+00,
         1.02571305e+01,   3.08244882e+01,   5.85276982e+01,
         8.32940947e+01,   1.06131490e+02,   1.32123434e+02,
         1.70992643e+02,   1.82781700e+02,   1.75773658e+02,
         1.74608856e+02,   1.82813491e+02,   1.75278616e+02,
         1.37499070e+02,   1.10361511e+02,   8.84823685e+01,
         6.45364083e+01,